# Classification NBA Model

## Configuration

In [34]:
from pathlib import Path

save_models_path = "/home/adrian_alvarez/Projects/NBA_over_under_predictor/models/classifier_autoglue_3/"
Path(save_models_path).mkdir(parents=True, exist_ok=True)


## Imports

In [35]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from autogluon.tabular import TabularPredictor
from nba_ou.data_preparation.missing_data.handle_missing_data import (
    apply_missing_policy,
    compute_and_save_train_medians,
)
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    confusion_matrix,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
)
from sklearn.model_selection import KFold, cross_validate, train_test_split
from xgboost import XGBClassifier


## Load Data

In [68]:
data_path = "/home/adrian_alvarez/Projects/NBA_over_under_predictor/data/train_data/"
name = "training_data_until_20260121_plus_0_5.csv"

path = data_path + name

df_stats = pd.read_csv(path)

dtype_dict = {col: str for col in df_stats.columns if "ID" in col.upper()}

df_stats = pd.read_csv(
    path,
    dtype=dtype_dict
)
df_stats['GAME_DATE'] = pd.to_datetime(df_stats['GAME_DATE']).dt.strftime('%Y-%m-%d')

/tmp/ipykernel_2991450/2366146840.py:6: DtypeWarning: Columns (205,206,207,248,249,250,291,292,293,334,335,336,377,378,379,420,421,422,616,617,618,659,660,661,702,703,704,745,746,747,788,789,790,831,832,833) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stats = pd.read_csv(path)
/tmp/ipykernel_2991450/2366146840.py:10: DtypeWarning: Columns (205,206,207,248,249,250,291,292,293,334,335,336,377,378,379,420,421,422,616,617,618,659,660,661,702,703,704,745,746,747,788,789,790,831,832,833) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stats = pd.read_csv(


In [69]:
df_stats= df_stats[df_stats['TOTAL_POINTS'] > 130]

In [70]:
df_stats.shape[0]

23774

In [71]:
df_stats

,TOTAL_OVER_UNDER_LINE,TEAM_ID_TEAM_HOME,TEAM_CITY_TEAM_HOME,TEAM_ABBREVIATION_TEAM_HOME,TEAM_NAME_TEAM_HOME,MATCHUP_TEAM_HOME,GAME_NUMBER_TEAM_HOME,TEAM_ID_TEAM_AWAY,TEAM_CITY_TEAM_AWAY,TEAM_ABBREVIATION_TEAM_AWAY,...,TRAVEL_RECENCY_RATIO_AWAY_2D_OVER_14D,REST_DAYS_DIFF_HOME_MINUS_AWAY,INJURY_PTS_SHARE_HOME,INJURY_PTS_SHARE_AWAY,STAR_PTS_PCT_DIFF_HOME_MINUS_AWAY,POSS_X_TSPCT_HOME,POSS_X_TSPCT_AWAY,IS_WEEKEND,MONTH,IS_US_HOLIDAY
0,236.4999,1610612766,Charlotte,CHA,Charlotte Hornets,CHA vs. CLE,44,1610612739,Cleveland,CLE,...,0.794552,1,0.133469,0.295323,-0.066251,57.210593,60.164427,0,1,0
1,225.9999,1610612758,Sacramento,SAC,Sacramento Kings,SAC vs. TOR,45,1610612761,Toronto,TOR,...,0.701086,0,0.280692,0.329806,-0.011903,56.125283,57.174706,0,1,0
2,232.9999,1610612740,New Orleans,NOP,New Orleans Pelicans,NOP vs. DET,46,1610612765,Detroit,DET,...,NaN,1,0.183386,0.314486,0.043431,56.721338,58.560329,0,1,0
3,225.4999,1610612749,Milwaukee,MIL,Milwaukee Bucks,MIL vs. OKC,43,1610612760,Oklahoma City,OKC,...,0.870530,0,0.315103,0.291012,-0.017535,60.746091,61.662553,0,1,0
4,219.4999,1610612752,New York,NYK,New York Knicks,NYK vs. BKN,44,1610612751,Brooklyn,BKN,...,0.794730,0,0.032962,0.082529,-0.001115,58.046030,50.774028,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23769,191.0000,1610612763,Memphis,MEM,Memphis Grizzlies,MEM vs. SAS,1,1610612759,San Antonio,SAS,...,NaN,-1,NaN,NaN,NaN,NaN,NaN,0,10,0
23770,183.5000,1610612740,New Orleans,NOH,New Orleans Pelicans,NOH vs. SAC,1,1610612758,Sacramento,SAC,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0
23771,203.0000,1610612747,Los Angeles,LAL,Los Angeles Lakers,LAL vs. HOU,1,1610612745,Houston,HOU,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0
23772,215.5000,1610612759,San Antonio,SAS,San Antonio Spurs,SAS vs. POR,1,1610612757,Portland,POR,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0


In [72]:
#count nas in TOTA_OVER_UNDER_LINE
nans = df_stats['TOTAL_OVER_UNDER_LINE'].isna().sum()
print(f"Number of NaNs in TOTAL_OVER_UNDER_LINE: {nans}")

Number of NaNs in TOTAL_OVER_UNDER_LINE: 1716


In [73]:
df_stats.dropna(subset=['TOTAL_OVER_UNDER_LINE'], inplace=True)
df_stats = df_stats[df_stats['TOTAL_OVER_UNDER_LINE'] > 100]

In [74]:
df_stats['SEASON_YEAR'].unique()

array([2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015,
       2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007])

In [75]:
#drop season 2007
df_stats = df_stats[df_stats['SEASON_YEAR'] != 2007]

In [76]:
df_to_train = df_stats.copy()

In [77]:
text_columns = df_to_train.select_dtypes(include=['object', 'string']).columns

# Print the names of the columns containing text
# print("Text columns:", list(text_columns))
cols_to_drop = ['SEASON_ID', 'GAME_ID', 'SEASON_TYPE',  'TEAM_ID_TEAM_HOME',
       'TEAM_ID_TEAM_AWAY','IS_OVERTIME']

cols_to_drop += [
    "MATCHUP_TEAM_HOME",
    "TEAM_CITY_TEAM_AWAY",
    "TEAM_ABBREVIATION_TEAM_AWAY",
    "TEAM_ABBREVIATION_TEAM_HOME",
    "TEAM_CITY_TEAM_HOME",
    "TEAM_NAME_TEAM_AWAY",
    "MATCHUP_TEAM_AWAY",
    "TEAM_NAME_TEAM_HOME"
]
cols_to_drop += list(text_columns)

cols_to_drop.pop(cols_to_drop.index('GAME_DATE'))

df_to_train.drop(columns=cols_to_drop, inplace=True, errors='ignore')


In [78]:
#Calculate number fo rows with at least 1 NA in total and in percentage
rows_with_na = df_to_train.isna().any(axis=1).sum()
total_rows = df_to_train.shape[0]
percentage_with_na = (rows_with_na / total_rows) * 100
print(f"Rows with at least one NA: {rows_with_na} out of {total_rows} ({percentage_with_na:.2f}%)")

Rows with at least one NA: 1852 out of 20845 (8.88%)


In [79]:
# Count NAs per column
na_counts = df_to_train.isna().sum()

# Get most common SEASON_YEAR for nulls in each column
most_common_season = []
for col in df_to_train.columns:
    if na_counts[col] > 0:
        # Get rows where this column is null
        null_rows = df_stats[df_stats[col].isna()]
        if len(null_rows) > 0 and 'SEASON_YEAR' in df_stats.columns:
            # Find most common SEASON_YEAR for these null rows
            common_season = null_rows['SEASON_YEAR'].mode()
            most_common_season.append(common_season.iloc[0] if len(common_season) > 0 else None)
        else:
            most_common_season.append(None)
    else:
        most_common_season.append(None)

na_counts_df = pd.DataFrame({
    'Column': na_counts.index,
    'NA_Count': na_counts.values,
    'NA_Percentage': (na_counts.values / len(df_to_train) * 100).round(2),
    'Most_Common_Season_Year': most_common_season
}).sort_values('NA_Count', ascending=False)

# Show only columns with NAs
na_counts_df[na_counts_df['NA_Count'] > 0]

,Column,NA_Count,NA_Percentage,Most_Common_Season_Year
515,TOTAL_OVER_UNDER_LINE_COMBINED_TREND_SLOPE_LAS...,623,2.99,2022.0
614,TRAVEL_RECENCY_RATIO_AWAY_2D_OVER_14D,562,2.70,2019.0
610,PTS_FORM_Z_AWAY_LAST5_VS_SEASON,561,2.69,2010.0
609,PTS_FORM_Z_HOME_LAST5_VS_SEASON,554,2.66,2022.0
497,DIFERENCE_POINTS_CONCEDED_VS_EXPECTED_BEFORE_A...,540,2.59,2008.0
...,...,...,...,...
529,REF_TRIO_DIFFERENCE_FROM_LINE_DIFF_BEFORE,22,0.11,2023.0
602,IMPLIED_PTS_HOME,11,0.05,2011.0
603,IMPLIED_PTS_AWAY,11,0.05,2011.0
532,MONEYLINE_TEAM_HOME,11,0.05,2011.0


In [80]:
# df_to_train.fillna(0, inplace=True)

In [81]:
print("Applying missing data policy...")
print(f"Data shape before handling missing data: {df_to_train.shape}")

Applying missing data policy...
Data shape before handling missing data: (20845, 623)


In [82]:
# Training time
train_medians = compute_and_save_train_medians(df_to_train)

# Prediction time
df_to_train, report = apply_missing_policy(
    df_to_train,
    train_medians=train_medians,
    current_total_line_col="TOTAL_OVER_UNDER_LINE",
    drop_mode="strict",
)


In [83]:
print(f"Data shape after handling missing data: {df_to_train.shape}")

Data shape after handling missing data: (20458, 623)


In [84]:
null_rows = df_to_train[df_to_train.isnull().any(axis=1)]
print(f"Total number of null values in the DataFrame: {null_rows.shape[0]}")
print(f"Percentage of null values in the DataFrame: {null_rows.shape[0] / df_to_train.shape[0] * 100:.2f}%")
print(f"Total number of rows in the DataFrame: {df_to_train.shape[0]}")

null_columns = df_to_train.columns[df_to_train.isnull().any()].tolist()
print(f"Columns with null values: {null_columns}")


Total number of null values in the DataFrame: 0
Percentage of null values in the DataFrame: 0.00%
Total number of rows in the DataFrame: 20458
Columns with null values: []


In [85]:
# df_to_train = df_to_train[df_to_train['GAME_DATE'] > "2010-12-15"]
# df_to_train = df_to_train[df_to_train['GAME_DATE'] <= "2019-07-15"]

In [86]:
df_to_train.drop(columns='GAME_DATE', inplace=True)


In [87]:
df_to_train['OVER_UNDER'] = np.where(df_to_train['TOTAL_POINTS'] >= df_to_train['TOTAL_OVER_UNDER_LINE'], 1, 0) # 1 if over, 0 if under

In [88]:
#count 1 and 0 in OVER_UNDER
df_to_train['OVER_UNDER'].value_counts()

OVER_UNDER
0    10678
1     9780
Name: count, dtype: int64

## Train / Test

In [89]:
X = df_to_train.drop(cols_to_drop + ['TOTAL_POINTS', 'OVER_UNDER', 'SEASON_YEAR'], axis=1, errors='ignore')
y = df_to_train['OVER_UNDER']

In [90]:
# Train / Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=16)

In [91]:
df_to_train['IS_TRAINING_DATA'] = False

# Mark True for the rows in the training set
df_to_train.loc[X_train.index, 'IS_TRAINING_DATA'] = True
# output_name = f"{data_path}/training_data_with_missing_data_handled_from_2004-10-01_to_2026-01-10_classifier.csv"
# df_to_train.to_csv(output_name, index=False)

In [92]:
print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
# Check number of coulmns
print(f"Number of columns in training set: {X_train.shape[1]}")
print(f"Number of columns in test set: {X_test.shape[1]}")

Training set size: 14320
Test set size: 6138
Number of columns in training set: 620
Number of columns in test set: 620


## Cross-validation

In [93]:
# Declare KFold
kf = KFold(n_splits=5, shuffle=True, random_state=16)

In [94]:
# Declare scores to be used
scoring = {
    'Acc': make_scorer(accuracy_score),
    'Prec': make_scorer(precision_score),
    'Rec': make_scorer(recall_score),
    'Bacc': make_scorer(balanced_accuracy_score),
}

In [95]:

def print_classification_metrics(cv_results, scoring):
    """
    Prints the mean cross-validation scores for each metric on train and validation folds.
    """
    for sc in scoring.keys():
        print(f"Train {sc}:", cv_results[f'train_{sc}'].mean().round(5))
        print(f"Validation {sc}:", cv_results[f'test_{sc}'].mean().round(5))
        print()
    

## Baseline

In [96]:
dummy_clf = DummyClassifier(strategy='most_frequent')
cv_results = cross_validate(
    dummy_clf, 
    X_train, 
    y_train, 
    cv=kf, 
    scoring=scoring, 
    return_train_score=True
)
dummy_clf.fit(X_train, y_train)
print("=== Dummy (Most Frequent) Classifier ===")
print_classification_metrics(cv_results, scoring)

/home/adrian_alvarez/Projects/NBA_over_under_predictor/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/adrian_alvarez/Projects/NBA_over_under_predictor/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/adrian_alvarez/Projects/NBA_over_under_predictor/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavi

=== Dummy (Most Frequent) Classifier ===
Train Acc: 0.52291
Validation Acc: 0.52291

Train Prec: 0.0
Validation Prec: 0.0

Train Rec: 0.0
Validation Rec: 0.0

Train Bacc: 0.5
Validation Bacc: 0.5



/home/adrian_alvarez/Projects/NBA_over_under_predictor/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Logistic Regression

In [97]:
lr = LogisticRegression(max_iter=1000)
cv_results = cross_validate(
    lr, 
    X_train, 
    y_train, 
    cv=kf, 
    scoring=scoring, 
    return_train_score=True,
    n_jobs=-1
)
lr.fit(X_train, y_train)


/home/adrian_alvarez/Projects/NBA_over_under_predictor/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/adrian_alvarez/Projects/NBA_over_under_predictor/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver option

LogisticRegression(max_iter=1000)

In [98]:
print("=== Logistic Regression ===")
print_classification_metrics(cv_results, scoring)

=== Logistic Regression ===
Train Acc: 0.68418
Validation Acc: 0.65098

Train Prec: 0.6758
Validation Prec: 0.6403

Train Rec: 0.64959
Validation Rec: 0.61306

Train Bacc: 0.68264
Validation Bacc: 0.64937



In [99]:
xgb_clf = XGBClassifier(
    max_depth=5,
    learning_rate=0.025,
    n_estimators=500,
    subsample=0.6,
    colsample_bytree=0.8,
    n_jobs=-1,
    random_state=16
)

cv_results = cross_validate(
    xgb_clf, 
    X_train, 
    y_train, 
    cv=kf, 
    scoring=scoring, 
    return_train_score=True,
    n_jobs=-1
)

xgb_clf.fit(X_train, y_train)

print("=== XGBClassifier ===")
print_classification_metrics(cv_results, scoring)

=== XGBClassifier ===
Train Acc: 0.91128
Validation Acc: 0.64846

Train Prec: 0.91881
Validation Prec: 0.64033

Train Rec: 0.89296
Validation Rec: 0.60135

Train Bacc: 0.91047
Validation Bacc: 0.64657



In [101]:
# Calcualte it in test set
y_pred = xgb_clf.predict(X_test)

#Predict test set with XGBClassifier
y_pred = xgb_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
balan_acc = balanced_accuracy_score(y_test, y_pred)
print("=== XGBClassifier Test Set Performance ===")
print(f"Accuracy: {accuracy:.5f}")
print(f"Precision: {precision:.5f}")    
print(f"Recall: {recall:.5f}")
print(f"F1 Score: {f1:.5f}")
print(f"Balanced Accuracy: {balan_acc:.5f}")

#groupby season_year and calculate accuracy per season
df_test = X_test.copy()
df_test['TRUE_LABEL'] = y_test
df_test['PRED_LABEL'] = y_pred
df_test['CORRECT_PREDICTION'] = np.where(df_test['TRUE_LABEL'] == df_test['PRED_LABEL'], 1, 0)
#merge 'SEASON_YEAR' from df_to_train
df_test = df_test.merge(df_to_train[['SEASON_YEAR']], left_index=True, right_index=True, how='left')
season_accuracy = df_test.groupby('SEASON_YEAR')['CORRECT_PREDICTION'].mean().reset_index()
season_accuracy.rename(columns={'CORRECT_PREDICTION': 'ACCURACY'}, inplace=True)
print("=== Test Set Accuracy by Season ===")
print(season_accuracy)

=== XGBClassifier Test Set Performance ===
Accuracy: 0.65445
Precision: 0.65332
Recall: 0.59769
F1 Score: 0.62427
Balanced Accuracy: 0.65229
=== Test Set Accuracy by Season ===
    SEASON_YEAR  ACCURACY
0          2008  0.727778
1          2009  0.716981
2          2010  0.717514
3          2011  0.648438
4          2012  0.707317
5          2013  0.719346
6          2014  0.686327
7          2015  0.706587
8          2016  0.747922
9          2017  0.620787
10         2018  0.684932
11         2019  0.673401
12         2020  0.643750
13         2021  0.660969
14         2022  0.582840
15         2023  0.505208
16         2024  0.493671
17         2025  0.465241


## AutoGluon (Hyperparameter Tuning)

In [128]:
predictor = TabularPredictor(
    label="target",
    problem_type="binary",
    eval_metric="balanced_accuracy",
    path=save_models_path,
).fit(
    train_data=X_train.assign(target=y_train),
    time_limit=360,
    presets="good_quality",
    hyperparameter_tune_kwargs="auto",
    num_cpus=10,
    hyperparameters={
        "XGB": {},  # only XGBoost models
    },
)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.3
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2
CPU Count:          12
Pytorch Version:    2.9.1+cu128
CUDA Version:       CUDA is not available
Memory Avail:       12.85 GB / 30.98 GB (41.5%)
Disk Space Avail:   25.20 GB / 464.35 GB (5.4%)
Presets specified: ['good_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack i

In [93]:
lb = predictor.leaderboard(silent=True)
lb


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.659945,balanced_accuracy,6.141328,1560.327056,0.001540,0.159979,3,False,14
1,XGBoost_BAG_L2/T2,0.659766,balanced_accuracy,5.526797,1513.529085,0.600929,105.322019,2,False,8
2,XGBoost_BAG_L2/T4,0.659519,balanced_accuracy,5.538859,1454.845058,0.612991,46.637992,2,False,10
3,XGBoost_BAG_L2/T1,0.659319,balanced_accuracy,5.625588,1540.722548,0.699719,132.515481,2,False,7
4,XGBoost_BAG_L2/T6,0.659293,balanced_accuracy,5.395520,1456.069732,0.469652,47.862666,2,False,12
5,XGBoost_BAG_L1/T4,0.658575,balanced_accuracy,0.725207,88.606333,0.725207,88.606333,1,False,4
6,WeightedEnsemble_L2,0.658575,balanced_accuracy,0.726597,88.817331,0.001390,0.210999,2,False,6
7,XGBoost_BAG_L2/T3,0.658041,balanced_accuracy,5.745912,1640.234595,0.820044,232.027529,2,False,9
8,XGBoost_BAG_L2/T7,0.655853,balanced_accuracy,5.723388,1589.924850,0.797520,181.717783,2,False,13
9,XGBoost_BAG_L1/T3,0.654524,balanced_accuracy,1.161263,375.429717,1.161263,375.429717,1,False,3


In [129]:
# Keep only the best model (+ its dependencies), delete the rest from disk
predictor.delete_models(models_to_keep="best", delete_from_disk=True)

# (Optional) shrink further by removing auxiliary artifacts not needed for prediction
predictor.save_space(remove_data=True)

# Save back to disk (same folder); after this, the directory is much smaller
predictor.save()

Deleting model XGBoost_BAG_L1/T1. All files under /home/adrian_alvarez/Projects/NBA_over_under_predictor/models/classifier_autoglue_3/models/XGBoost_BAG_L1/T1 will be removed.
Deleting model WeightedEnsemble_L2. All files under /home/adrian_alvarez/Projects/NBA_over_under_predictor/models/classifier_autoglue_3/models/WeightedEnsemble_L2 will be removed.
Deleting model XGBoost_BAG_L2/T1. All files under /home/adrian_alvarez/Projects/NBA_over_under_predictor/models/classifier_autoglue_3/models/XGBoost_BAG_L2/T1 will be removed.
Deleting model WeightedEnsemble_L3. All files under /home/adrian_alvarez/Projects/NBA_over_under_predictor/models/classifier_autoglue_3/models/WeightedEnsemble_L3 will be removed.
Deleting model WeightedEnsemble_L2_FULL. All files under /home/adrian_alvarez/Projects/NBA_over_under_predictor/models/classifier_autoglue_3/models/WeightedEnsemble_L2_FULL will be removed.
Deleting model XGBoost_BAG_L2/T1_FULL. All files under /home/adrian_alvarez/Projects/NBA_over_unde

## Test Set Evaluation (Best Model)

In [95]:

y_test_pred = predictor.predict(X_test)

metrics = {
    "accuracy": accuracy_score(y_test, y_test_pred),
    "balanced_accuracy": balanced_accuracy_score(y_test, y_test_pred),
    "precision": precision_score(y_test, y_test_pred),
    "recall": recall_score(y_test, y_test_pred),
    "f1": f1_score(y_test, y_test_pred),
    "error_rate": 1 - accuracy_score(y_test, y_test_pred),
}

pd.Series(metrics).round(4)


accuracy             0.6546
balanced_accuracy    0.6535
precision            0.6623
recall               0.6012
f1                   0.6303
error_rate           0.3454
dtype: float64